# logging.py 테스트

## 목적
- `src/utils/logging.py` 정상 작동 확인
- 각 기능별 테스트 수행

## 테스트 항목
1. 환경 설정 및 모듈 임포트
2. setup_logging() 함수 테스트
3. get_logger() 함수 테스트
4. 로그 레벨별 출력 테스트
5. 로그 파일 생성 확인
6. logging_config.py와 비교 테스트

---
## 0. 환경 설정

In [1]:
import sys
from pathlib import Path

# 프로젝트 루트 설정
project_root = Path.cwd()
while project_root.name != 'codeit_ad_smallbiz' and project_root.parent != project_root:
    project_root = project_root.parent

if project_root.name == 'codeit_ad_smallbiz':
    sys.path.insert(0, str(project_root))
    print(f"프로젝트 루트 설정: {project_root}")
else:
    print("codeit_ad_smallbiz 폴더를 찾을 수 없습니다.")
    print(f"현재 위치: {Path.cwd()}")

프로젝트 루트 설정: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz


In [3]:
# 테스트용 로그 디렉토리 생성
import os

log_dir = project_root / "logs"
log_dir.mkdir(exist_ok=True)
print(f"로그 디렉토리: {log_dir}")
print(f"디렉토리 존재: {log_dir.exists()}")

로그 디렉토리: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz\logs
디렉토리 존재: True


---
## 1. logging.py 모듈 임포트 테스트

In [4]:
# logging.py 모듈 임포트
try:
    from src.utils.logging import (
        setup_logging,
        get_logger,
        RUN_ID,
        RUN_LOG_FILE,
        UVICORN_AVAILABLE,
        DEFAULT_FORMAT,
        DEFAULT_DATEFMT
    )
    print("[PASS] logging.py 모듈 임포트 성공")
    print(f"  - UVICORN_AVAILABLE: {UVICORN_AVAILABLE}")
    print(f"  - DEFAULT_FORMAT: {DEFAULT_FORMAT}")
    print(f"  - DEFAULT_DATEFMT: {DEFAULT_DATEFMT}")
except ImportError as e:
    print(f"[FAIL] logging.py 모듈 임포트 실패: {e}")

[PASS] logging.py 모듈 임포트 성공
  - UVICORN_AVAILABLE: False
  - DEFAULT_FORMAT: %(asctime)s.%(msecs)03d - %(levelname)s - %(name)s - %(message)s
  - DEFAULT_DATEFMT: %Y-%m-%d %H:%M:%S


---
## 2. setup_logging() 함수 테스트

In [5]:
import logging

# 기존 핸들러 초기화 (테스트 반복 실행 시 필요)
logging.root.handlers = []

# setup_logging 호출 (프로젝트 logs 폴더 사용)
try:
    setup_logging(log_dir=str(log_dir))
    print("[PASS] setup_logging() 호출 성공")
except Exception as e:
    print(f"[FAIL] setup_logging() 호출 실패: {e}")

2026-01-22 16:51:38.607 - I - src.utils.logging - Logging initialized: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz\logs\20260122_165138.log
[PASS] setup_logging() 호출 성공


In [6]:
# RUN_ID와 RUN_LOG_FILE 확인
from src.utils.logging import RUN_ID, RUN_LOG_FILE

print(f"RUN_ID: {RUN_ID}")
print(f"RUN_LOG_FILE: {RUN_LOG_FILE}")

if RUN_ID and RUN_LOG_FILE:
    print("[PASS] RUN_ID, RUN_LOG_FILE 설정됨")
else:
    print("[FAIL] RUN_ID 또는 RUN_LOG_FILE이 None")

RUN_ID: 20260122_165138
RUN_LOG_FILE: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz\logs\20260122_165138.log
[PASS] RUN_ID, RUN_LOG_FILE 설정됨


In [7]:
# Root 로거 핸들러 확인
root_logger = logging.getLogger()

print(f"Root 로거 레벨: {logging.getLevelName(root_logger.level)}")
print(f"핸들러 개수: {len(root_logger.handlers)}")

for i, handler in enumerate(root_logger.handlers):
    print(f"  [{i}] {type(handler).__name__}")
    if hasattr(handler, 'baseFilename'):
        print(f"      파일: {handler.baseFilename}")

if len(root_logger.handlers) >= 2:
    print("[PASS] 콘솔 + 파일 핸들러 설정됨")
else:
    print("[WARN] 핸들러가 2개 미만")

Root 로거 레벨: INFO
핸들러 개수: 2
  [0] StreamHandler
  [1] RotatingFileHandler
      파일: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz\logs\20260122_165138.log
[PASS] 콘솔 + 파일 핸들러 설정됨


---
## 3. get_logger() 함수 테스트

In [8]:
# get_logger() 테스트
logger = get_logger(__name__)

print(f"로거 이름: {logger.name}")
print(f"로거 레벨: {logging.getLevelName(logger.level)}")
print(f"effective 레벨: {logging.getLevelName(logger.getEffectiveLevel())}")

if logger:
    print("[PASS] get_logger() 동작 확인")
else:
    print("[FAIL] get_logger() 실패")

로거 이름: __main__
로거 레벨: NOTSET
effective 레벨: INFO
[PASS] get_logger() 동작 확인


In [9]:
# 여러 모듈용 로거 생성 테스트
logger_backend = get_logger("src.backend.services")
logger_chatbot = get_logger("src.backend.chatbot")
logger_generation = get_logger("src.generation.image")

print("생성된 로거들:")
print(f"  - {logger_backend.name}")
print(f"  - {logger_chatbot.name}")
print(f"  - {logger_generation.name}")
print("[PASS] 모듈별 로거 생성 성공")

생성된 로거들:
  - src.backend.services
  - src.backend.chatbot
  - src.generation.image
[PASS] 모듈별 로거 생성 성공


---
## 4. 로그 레벨별 출력 테스트

In [10]:
# 로그 레벨별 출력 테스트
test_logger = get_logger("test.levels")

print("=" * 60)
print("로그 레벨별 출력 테스트 (INFO 이상만 출력됨)")
print("=" * 60)

test_logger.debug("DEBUG 메시지 - 이 메시지는 기본 설정에서 보이지 않음")
test_logger.info("INFO 메시지 - 일반 정보")
test_logger.warning("WARNING 메시지 - 경고")
test_logger.error("ERROR 메시지 - 에러 발생")
test_logger.critical("CRITICAL 메시지 - 심각한 오류")

print("=" * 60)
print("[PASS] 로그 출력 테스트 완료")

로그 레벨별 출력 테스트 (INFO 이상만 출력됨)
2026-01-22 16:51:59.473 - I - test.levels - INFO 메시지 - 일반 정보
2026-01-22 16:51:59.475 - W - test.levels - WARNING 메시지 - 경고
2026-01-22 16:51:59.476 - E - test.levels - ERROR 메시지 - 에러 발생
2026-01-22 16:51:59.479 - C - test.levels - CRITICAL 메시지 - 심각한 오류
[PASS] 로그 출력 테스트 완료


In [10]:
# DEBUG 레벨로 변경하여 테스트
print("=" * 60)
print("DEBUG 레벨 활성화 테스트")
print("=" * 60)

# 임시로 DEBUG 레벨 설정
original_level = logging.root.level
logging.root.setLevel(logging.DEBUG)

test_logger.debug("DEBUG 메시지 - 이제 보임!")
test_logger.info("INFO 메시지")

# 원래 레벨로 복원
logging.root.setLevel(original_level)

print("=" * 60)
print(f"[PASS] DEBUG 레벨 테스트 완료 (레벨 복원: {logging.getLevelName(original_level)})")

DEBUG 레벨 활성화 테스트
2026-01-21 15:49:02.848 - D - test.levels - DEBUG 메시지 - 이제 보임!
2026-01-21 15:49:02.850 - I - test.levels - INFO 메시지
[PASS] DEBUG 레벨 테스트 완료 (레벨 복원: INFO)


---
## 5. 로그 파일 생성 확인

In [11]:
# 로그 파일 존재 확인
from src.utils.logging import RUN_LOG_FILE

log_file_path = Path(RUN_LOG_FILE)

print(f"로그 파일 경로: {log_file_path}")
print(f"파일 존재: {log_file_path.exists()}")

if log_file_path.exists():
    print(f"파일 크기: {log_file_path.stat().st_size} bytes")
    print("[PASS] 로그 파일 생성됨")
else:
    print("[FAIL] 로그 파일이 생성되지 않음")

로그 파일 경로: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz\logs\20260121_154725.log
파일 존재: True
파일 크기: 599 bytes
[PASS] 로그 파일 생성됨


In [12]:
# 로그 파일 내용 확인 (최근 20줄)
if log_file_path.exists():
    print("=" * 60)
    print("로그 파일 내용 (최근 20줄)")
    print("=" * 60)
    
    with open(log_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[-20:]:
            print(line.rstrip())
    
    print("=" * 60)
    print(f"총 {len(lines)}줄")
else:
    print("로그 파일이 없습니다.")

로그 파일 내용 (최근 20줄)
2026-01-21 15:47:25.491 - I - src.utils.logging - Logging initialized: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz\logs\20260121_154725.log
2026-01-21 15:48:48.806 - I - test.levels - INFO 메시지 - 일반 정보
2026-01-21 15:48:48.808 - W - test.levels - WARNING 메시지 - 경고
2026-01-21 15:48:48.810 - E - test.levels - ERROR 메시지 - 에러 발생
2026-01-21 15:48:48.813 - C - test.levels - CRITICAL 메시지 - 심각한 오류
2026-01-21 15:49:02.848 - D - test.levels - DEBUG 메시지 - 이제 보임!
2026-01-21 15:49:02.850 - I - test.levels - INFO 메시지
총 7줄


In [13]:
# logs 폴더 내 모든 로그 파일 목록
print("logs 폴더 내 파일 목록:")
print("-" * 40)

for f in sorted(log_dir.glob("*.log")):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name} ({size_kb:.1f} KB)")

print("-" * 40)

logs 폴더 내 파일 목록:
----------------------------------------
  20260121_154725.log (0.6 KB)
----------------------------------------


---
## 6. 실제 사용 시나리오 테스트

In [15]:
# 실제 서비스 모듈처럼 사용하는 예시
from src.utils.logging import setup_logging, get_logger

# 모듈 시뮬레이션
class MockChatbotService:
    def __init__(self):
        self.logger = get_logger(self.__class__.__name__)
        self.logger.info("ChatbotService 초기화")
    
    def process_message(self, message: str):
        self.logger.info(f"메시지 처리 시작: {message[:20]}...")
        try:
            # 처리 로직 시뮬레이션
            result = f"응답: {message}"
            self.logger.info("메시지 처리 완료")
            return result
        except Exception as e:
            self.logger.error(f"메시지 처리 실패: {e}")
            raise

# 테스트 실행
print("=" * 60)
print("MockChatbotService 테스트")
print("=" * 60)

service = MockChatbotService()
result = service.process_message("안녕하세요, 광고 문구를 만들어주세요.")
print(f"결과: {result}")

print("=" * 60)
print("[PASS] 실제 사용 시나리오 테스트 완료")

MockChatbotService 테스트
2026-01-21 16:29:22.267 - I - MockChatbotService - ChatbotService 초기화
2026-01-21 16:29:22.269 - I - MockChatbotService - 메시지 처리 시작: 안녕하세요, 광고 문구를 만들어주세요...
2026-01-21 16:29:22.270 - I - MockChatbotService - 메시지 처리 완료
결과: 응답: 안녕하세요, 광고 문구를 만들어주세요.
[PASS] 실제 사용 시나리오 테스트 완료


In [16]:
# 예외 로깅 테스트
exception_logger = get_logger("test.exception")

print("=" * 60)
print("예외 로깅 테스트")
print("=" * 60)

try:
    # 의도적 예외 발생
    result = 1 / 0
except ZeroDivisionError as e:
    exception_logger.error(f"예외 발생: {e}")
    exception_logger.exception("상세 스택 트레이스:")  # 스택 트레이스 포함

print("=" * 60)
print("[PASS] 예외 로깅 테스트 완료")

예외 로깅 테스트
2026-01-21 16:29:28.096 - E - test.exception - 예외 발생: division by zero
2026-01-21 16:29:28.098 - E - test.exception - 상세 스택 트레이스:
Traceback (most recent call last):
  File "C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23060\1464741100.py", line 10, in <module>
    result = 1 / 0
             ~~^~~
ZeroDivisionError: division by zero
[PASS] 예외 로깅 테스트 완료


---
## 7. 테스트 결과 요약

In [17]:
# 테스트 결과 요약
from src.utils.logging import RUN_ID, RUN_LOG_FILE, UVICORN_AVAILABLE

print("=" * 60)
print("테스트 결과 요약")
print("=" * 60)

results = {
    "모듈 임포트": True,
    "setup_logging() 호출": RUN_ID is not None,
    "get_logger() 동작": True,
    "로그 파일 생성": Path(RUN_LOG_FILE).exists() if RUN_LOG_FILE else False,
    "콘솔 출력": True,
    "uvicorn 포매터": UVICORN_AVAILABLE,
}

all_passed = True
for test_name, passed in results.items():
    status = "PASS" if passed else "FAIL"
    symbol = "O" if passed else "X"
    print(f"  [{symbol}] {test_name}: {status}")
    if not passed:
        all_passed = False

print("=" * 60)
if all_passed:
    print("모든 테스트 통과!")
else:
    print("일부 테스트 실패 - 위 결과 확인 필요")

print(f"\n로그 파일 위치: {RUN_LOG_FILE}")

테스트 결과 요약
  [O] 모듈 임포트: PASS
  [O] setup_logging() 호출: PASS
  [O] get_logger() 동작: PASS
  [O] 로그 파일 생성: PASS
  [O] 콘솔 출력: PASS
  [X] uvicorn 포매터: FAIL
일부 테스트 실패 - 위 결과 확인 필요

로그 파일 위치: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz\logs\20260121_154725.log


---
## 참고사항

### 현재 logging.py 개선이 필요한 부분
1. **로그 디렉토리 하드코딩**: `/mnt/logs` → Windows에서 동작 안 함
2. **파일 로테이션 없음**: 로그 파일 크기 무한 증가 가능
3. **로그 레벨 고정**: `INFO` 레벨 고정, 환경 변수 연동 없음

### Pull Request에 logging.py 변경 내용 요약
1. logging.py 현재 구조 검토
장점: uvicorn 선택적 의존성 처리, 실행 단위 로그 파일 생성, 콘솔/파일 분리
개선 필요: 하드코딩된 경로(/mnt/logs), 고정된 로그 레벨, 파일 로테이션 없음
2. logging_config.py에서 가져올 기능
RotatingFileHandler - 로그 파일 크기 제한 및 자동 롤오버
CompactFormatter - 로그 레벨 한 글자 표시 (INFO → I)
ColoredFormatter - uvicorn 없이도 터미널 색상 지원
환경 변수 연동 (LOG_LEVEL, LOG_DIR)
편의 함수 (log_info(), log_error() 등)
3. 수정 방안
플랫폼 독립적 로그 디렉토리 탐지
RotatingFileHandler로 10MB/5백업 제한
커스텀 포매터 통합
환경 변수 기반 설정
4. 활용법
FastAPI/uvicorn 및 일반 스크립트 환경별 사용 예시
모듈별 로거 사용 권장 패턴
마이그레이션 가이드 포함

- 원본 내용은 `260121_logging_개선방향.md` 문서 참조